In [1]:
import pandas as pd

uid_check = ['아파트1-104-1006']


In [2]:
total_ecv = pd.DataFrame(columns=['UECV', 'SORTECV', "UCPDV", "SORTCPDV"])

In [3]:
# uid_check.remove('아파트3-103-312')
len(uid_check)

1

In [ ]:
from modules.UCKMeans import UCKMeans as UC
from modules.UCOSKMeans import UCOSKMeans as UCOS
from modules.SortMeans import SortMeans as SM

# Mongo 연결
from pymongo import MongoClient as mc
import random
import pprint as pp

#  print(uid_check)
# custom module
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)

keti_pr_db = client.keti_pattern_recognition
household_col = keti_pr_db.household_info

for seq in range(0, 100):
    total = household_col.find({"uid": { "$nin" : uid_check }}).count()
    while True:
        db_datas = list(household_col.\
                        find({
                            "uid": { "$nin" : uid_check }
                            }).skip(random.randrange(0,total)).limit(1))
        if len(db_datas) == 0 :
            continue
        else:
            db_datas = db_datas[0]
            break;
    # db_datas = household_col.\
    #                 find_one({
    #                     "uid": "아파트4-103-606"
    #                 })


    # require module
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from datetime import datetime as dt

    sns.set(rc={'figure.figsize': (15.7, 13.27)})
    plt.rcParams['figure.figsize'] = 15.7,13.27
    plt.rcParams['font.family'] = 'AppleGothic'

    # custom module
    %load_ext autoreload
    %autoreload 2
    %reload_ext autoreload

    import custom.supports as sp
    import custom.mapgenerator as mg
    import custom.dateutils as du

    SEASONS = {
        "봄" : [3, 4, 5],
        "여름" : [6, 7, 8],
        "가을" : [9, 10, 11],
        "겨울" : [12, 1, 2]
    }

    SEASONSARR = ['봄','여름','가을','겨울']

    '''
     0 : 봄
     1 : 여름
     2 : 가을
     3 : 겨울
    '''

    DAYARR = ["월", "화", "수", "목", "금", "토", "일"]

    uid, timeslot = db_datas['uid'], db_datas['timeslot']
    uid_check.append(uid)

    print("{}:{} pattern recognition".format(seq,uid))

    datelist = [
        dt.strptime(ts['time'], "%Y-%m-%d T%H:%M %z").date()
        for ts in timeslot
    ]
    datelist = list(set(datelist))
    datelist.sort()

    ts_datas = {}
    start_idx = 0
    end_idx = 96
    enl = 1000

    for date in datelist:
        ts_datas[date] = [ ts['power'] * enl for ts in timeslot[start_idx:end_idx]]
        start_idx = end_idx
        end_idx = end_idx + 96

    ts_datas = pd.DataFrame(ts_datas).T
    datas = ts_datas.reset_index().copy()

    datas.rename(columns={"index": "date"}, inplace=True)
    datas['date'] = pd.to_datetime(datas['date'])
    datas['month'] = [dt.month for dt in datas['date']]
    datas = [
        datas[(datas['month'].isin(SEASONS[season]))].copy()
        for season in SEASONSARR
    ]

    bak_datas = datas.copy()

    season_datas = {
        "봄": datas[0].copy(),
        "여름": datas[1].copy(),
        "가을": datas[2].copy(),
        "겨울": datas[3].copy()
    }

    # merge timeslot
    season = "가을"

    timeslot = 1
    merge_size = timeslot * 4
    merge_datas = season_datas[season].copy()
    merge_datas = merge_datas[
        merge_datas.columns.difference(['month', 'date'])
    ]
    merge_datas.index = season_datas[season]['date']

    tmp = pd.DataFrame()
    start_idx = 0
    end_idx = 96
    for date in merge_datas.index:
        in_arr = []
        og_arr = merge_datas.loc[date].values
        for merging in range(0, int(end_idx / merge_size)):
            in_arr.append(og_arr[
                (merging) * merge_size
                :
                (merging + 1) * merge_size
            ].sum())
    #     logging
    #     print("date end : {}, size: {}".format(date, len(in_arr)))

        tmp[date] = in_arr
    merge_datas = tmp.copy()

    merge_datas
    
    
    uc = UC(merge_datas, K=10)
    uc.run()
    
#     ucos = UCOS(merge_datas, K=10)
#     ucos.run()

    sm = SM(merge_datas, K=10)
    sm.run()
    
    total_ecv.loc[uid] = [
        uc.get_ECV(),
        sm.get_ECV(),
        uc.get_CPDV(),
        sm.get_CPDV()
    ]
    
    print("now : {}".format(total_ecv.mean(axis=0)))
    
client.close()
total_ecv

<ipython-input-4-3d1ad5e7f25d>:23: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0:아파트4-103-1503 pattern recognition
---UCKMeans Init---
K:10
---Go UCMeans---
[]
-------Remove One Pattern Success-------
  Target length 91 -> 91
DIS_INFO {'Q1': 0.1353280918673671, 'Q3': 0.38483053133171796, 'IQR': 0.24950243946435086, 'step': 0.3742536591965263}
DatetimeIndex(['2018-09-24', '2018-10-29', '2018-11-26'], dtype='datetime64[ns]', freq=None)
-------Remove Outlier Success-------
  Target length 91 -> 88
---Now 0---
TSS: 40158131.44318183, WSS: 55839298.0, ECV: -39.048546317462105, CPDV: 0.9394820836273456
---Now 1---
TSS: 40158131.44318183, WSS: 24704634.783246633, ECV: 38.481612825536324, CPDV: 0.9737644139539264
---Now 2---
TSS: 40158131.44318183, WSS: 23594436.066535946, ECV: 41.246180490447394, CPDV: 0.9748745669994358
---Now 3---
TSS: 40158131.44318183, WSS: 23004763.83679557, ECV: 42.71455615572126, CPDV: 0.9753848171574563
---Now 4---
TSS: 40158131.44318183, WSS: 22398186.007979

<ipython-input-4-3d1ad5e7f25d>:23: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


---UCKMeans Init---
K:10
---Go UCMeans---
[]
-------Remove One Pattern Success-------
  Target length 91 -> 91
DIS_INFO {'Q1': 0.13094737237417187, 'Q3': 0.3018835266430557, 'IQR': 0.1709361542688838, 'step': 0.2564042314033257}
DatetimeIndex(['2018-10-20', '2018-10-23'], dtype='datetime64[ns]', freq=None)
-------Remove Outlier Success-------
  Target length 91 -> 89
---Now 0---
TSS: 24308064.853932597, WSS: 32479645.0, ECV: -33.61674487529349, CPDV: 0.9670330622220539
---Now 1---
TSS: 24308064.853932597, WSS: 15139856.507801102, ECV: 37.71673475952673, CPDV: 0.9819836571674203
---Now 2---
TSS: 24308064.853932597, WSS: 14588700.68988585, ECV: 39.984113184042016, CPDV: 0.9825636032060094
---Now 3---
TSS: 24308064.853932597, WSS: 14388984.358730154, ECV: 40.80571840994458, CPDV: 0.982726692688095
---Now 4---
TSS: 24308064.853932597, WSS: 14388984.358730154, ECV: 40.80571840994458, CPDV: 0.982726692688095
---SortMeans Init---
K:10
---Go SortMeans---
[]
-------Remove One Pattern Success---

<ipython-input-4-3d1ad5e7f25d>:23: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


...

---UCKMeans Init---
K:10
---Go UCMeans---
[]
-------Remove One Pattern Success-------
  Target length 91 -> 91
DIS_INFO {'Q1': 0.09982310060048653, 'Q3': 0.3122829354366603, 'IQR': 0.21245983483617376, 'step': 0.31868975225426066}
DatetimeIndex(['2018-09-03', '2018-10-02', '2018-11-25'], dtype='datetime64[ns]', freq=None)
-------Remove Outlier Success-------
  Target length 91 -> 88
---Now 0---
TSS: 20613083.579545446, WSS: 24251594.0, ECV: -17.6514610558562, CPDV: 0.9498231036267378
---Now 1---
TSS: 20613083.579545446, WSS: 13325108.000761569, ECV: 35.35606669744358, CPDV: 0.9729993381481218
---Now 2---
TSS: 20613083.579545446, WSS: 13069841.66885216, ECV: 36.59443712816706, CPDV: 0.9736361764680305
---Now 3---
TSS: 20613083.579545446, WSS: 12912022.508234126, ECV: 37.360063289866794, CPDV: 0.9737202418945877
---Now 4---
TSS: 20613083.579545446, WSS: 12757865.383878969, ECV: 38.10792385987938, CPDV: 0.9741218467574574
---Now 5---
TSS: 20613083.579545446, WSS: 12741970.373914924, ECV: 

<ipython-input-4-3d1ad5e7f25d>:23: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


---UCKMeans Init---
K:10
---Go UCMeans---
[]
-------Remove One Pattern Success-------
  Target length 91 -> 91
DIS_INFO {'Q1': 0.12110061449881379, 'Q3': 0.3791846385570248, 'IQR': 0.25808402405821096, 'step': 0.38712603608731644}
DatetimeIndex(['2018-09-08', '2018-09-24', '2018-10-14', '2018-11-30'], dtype='datetime64[ns]', freq=None)
-------Remove Outlier Success-------
  Target length 91 -> 87
---Now 0---
TSS: 181553302.04597703, WSS: 296694059.0, ECV: -63.41980875944875, CPDV: 0.8463624206488738
---Now 1---
TSS: 181553302.04597703, WSS: 102680996.70885701, ECV: 43.4430574648244, CPDV: 0.9352247304658997
---Now 2---
TSS: 181553302.04597703, WSS: 98936850.54739437, ECV: 45.50534227004071, CPDV: 0.9374312866892931
---Now 3---
TSS: 181553302.04597703, WSS: 97423699.82721789, ECV: 46.33878936415816, CPDV: 0.9380856314797079
---Now 4---
TSS: 181553302.04597703, WSS: 96729162.83318928, ECV: 46.721342028418, CPDV: 0.9386077678102726
---Now 5---
TSS: 181553302.04597703, WSS: 92697381.016198

<ipython-input-4-3d1ad5e7f25d>:23: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


---UCKMeans Init---
K:10
---Go UCMeans---
[]
-------Remove One Pattern Success-------
  Target length 91 -> 91
DIS_INFO {'Q1': 0.0685302811566531, 'Q3': 0.17695574448470652, 'IQR': 0.10842546332805342, 'step': 0.16263819499208013}
DatetimeIndex(['2018-09-23', '2018-10-10', '2018-10-11', '2018-11-11'], dtype='datetime64[ns]', freq=None)
-------Remove Outlier Success-------
  Target length 91 -> 87
---Now 0---
TSS: 49439392.04597703, WSS: 56552552.0, ECV: -14.387636375884139, CPDV: 0.9424129461109487
---Now 1---
TSS: 49439392.04597703, WSS: 24558885.035520338, ECV: 50.325268942058656, CPDV: 0.9654891551855049
---Now 2---
TSS: 49439392.04597703, WSS: 23660088.897935655, ECV: 52.143244650070656, CPDV: 0.9663957322191605
---Now 3---
TSS: 49439392.04597703, WSS: 23405344.734600123, ECV: 52.658510216238284, CPDV: 0.9666613751183767
---Now 4---
TSS: 49439392.04597703, WSS: 23362061.76932234, ECV: 52.74605774359771, CPDV: 0.9667118685719442
---Now 5---
TSS: 49439392.04597703, WSS: 23362061.7693

In [ ]:
total_ecv["ecv_win"] = \
    ["UECV" if total_ecv.loc[idx]['UECV'] > total_ecv.loc[idx]['SORTECV'] else "SORTECV" for idx in total_ecv.index]
total_ecv["cpdv_win"] = \
    ["UCPDV" if total_ecv.loc[idx]['UCPDV'] > total_ecv.loc[idx]['SORTCPDV'] else "SORTCPDV" for idx in total_ecv.index]

total_ecv["ecv_diff"] = \
    [(total_ecv.loc[idx]['UECV'] - total_ecv.loc[idx]['SORTECV'])
         if total_ecv.loc[idx]['UECV'] > total_ecv.loc[idx]['SORTECV'] else
             (total_ecv.loc[idx]['SORTECV'] - total_ecv.loc[idx]['UECV']) for idx in total_ecv.index]
total_ecv["cpdv_diff"] = \
    [(total_ecv.loc[idx]['UCPDV'] - total_ecv.loc[idx]['SORTCPDV'])
         if total_ecv.loc[idx]['UCPDV'] > total_ecv.loc[idx]['SORTCPDV'] else
             (total_ecv.loc[idx]['SORTCPDV'] - total_ecv.loc[idx]['UCPDV']) for idx in total_ecv.index]

total_ecv

In [ ]:
print("""---Uclidean Based Diff Max---
    ECV Win Count : {}
    ECV Diff Max : {}
    CPDV Win Count : {}
    CPDV Diff Max : {}
""".format(
total_ecv.groupby(total_ecv['ecv_win']).count().loc['UECV'].values[0],
total_ecv[
    total_ecv['ecv_win'] == "UECV"
]['ecv_diff'].max(),
total_ecv.groupby(total_ecv['cpdv_win']).count().loc['UCPDV'].values[0],
total_ecv[
    total_ecv['cpdv_win'] == "UCPDV"
]['cpdv_diff'].max()))

print("""---Uclidean + Cosine Based (Sort Based) Diff Max---
    ECV Win Count : {}
    ECV Diff Max : {}
    CPDV Win Count : {}
    CPDV Diff Max : {}
""".format(
total_ecv.groupby(total_ecv['ecv_win']).count().loc['SORTECV'].values[0],
total_ecv[
    total_ecv['ecv_win'] == "SORTECV"
]['ecv_diff'].max(),
total_ecv.groupby(total_ecv['cpdv_win']).count().loc['SORTCPDV'].values[0],
total_ecv[
    total_ecv['cpdv_win'] == "SORTCPDV"
]['cpdv_diff'].max()))

In [ ]:
total_ecv[
    total_ecv['ecv_win'] == "UECV"
].sort_values(by=['ecv_diff'], ascending=False)